In [12]:
import pandas as pd
from collections import Counter
import plotly.express as px

In [2]:
df = pd.read_csv('Datasets/population.csv')

In [14]:
# Neighborhood Wise
Districts = dict()
for index, row in df.iterrows():
    if row.loc['District.Name'] in Districts:
        Districts[row['District.Name']].update([row['Age']] * row['Number'])
    else:
        Districts[row['District.Name']] = Counter([row['Age']] * row['Number'])

ageDistribution_df = pd.DataFrame(Districts)

In [20]:
DistrictName = 'Ciutat Vella'
px.bar(ageDistribution_df[DistrictName], x=ageDistribution_df.axes[0], y=(ageDistribution_df[DistrictName]/ageDistribution_df[DistrictName].sum())*100, labels={'index': 'Ages', 'y': 'Percentage'})